# 【問題1】機械翻訳の実行とコードリーディング
以下のサンプルコードは、短い英語からフランス語への変換を行うものです。これを動かしてください。
その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめてください。以下のようにどこからどこの行が何をしているかを記述してください。

In [33]:
# レイヤー層のインポート
# from __future__ import print_functionは3系の機能を2系で使う事が可能でprintに言及している
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # バッチ
epochs = 100  # エポック
latent_dim = 256  # エンコーダーの次元 
num_samples = 10000 # サンプル数

# パス
data_path = '/Users/yuki.tatsuoka/diveintocode/week13~15課題/week14_課題/fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n') # 文章単位で区切る

# 文章１万を抽出
for line in lines[: min(num_samples, len(lines) - 1)]:
    # タブ単位で区切ってEOSにする
    input_text, target_text, _ = line.split('\t') 
    '''
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    '''
    # 細かい文字をappendで追加する
    target_text = '\t' + target_text + '\n' # '\n'＝ endシーケンス
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# 文字の長さを保存
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 学習用のエンコーダ、デコーダデータとラベルデータを用意
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # デコーダの推論の時系列
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# Seq2Seqの層の作成（エンコーダ）
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 途中経過のencoder_outputsを今回は採用しない
encoder_states = [state_h, state_c]

# Seq2Seqの層の作成（デコーダ）
# 基本エンコーダと同じ
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

# 今回は途中経過を推論に活かす
decoder_outputs = decoder_dense(decoder_outputs)

# functionalAPIに入れる
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 学習 
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# 保存
model.save('s2s.h5')

# デコーダを増やす処理
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# inputとtargetの値を保存
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # 予測 
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # EOSを設定
    target_seq[0, 0, target_token_index['\t']] = 1.

    # バッチシーケンスのループ
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # トークンのサンプル
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 最長か、endで終了する
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # ターゲットシーケンスの更新
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59
Epoch 1/100
125/125 [==============================] - 21s 153ms/step - loss: 1.0388 - val_loss: 1.0769
Epoch 2/100
125/125 [==============================] - 21s 167ms/step - loss: 0.9245 - val_loss: 1.0480
Epoch 3/100
125/125 [==============================] - 22s 176ms/step - loss: 0.8770 - val_loss: 0.9532
Epoch 4/100
125/125 [==============================] - 22s 173ms/step - loss: 0.8346 - val_loss: 0.9159
Epoch 5/100
125/125 [==============================] - 22s 178ms/step - loss: 0.8006 - val_loss: 0.8866
Epoch 6/100
125/125 [==============================] - 25s 197ms/step - loss: 0.7751 - val_loss: 0.8823
Epoch 7/100
125/125 [==============================] - 21s 171ms/step - loss: 0.7503 - val_loss: 0.8460
Epoch 8/100
125/125 [==============================] - 23s 188ms/step - loss: 0.7268 - val_loss: 0.8701
Epo

# 【問題2】イメージキャプショニングの学習済みモデルの実行
上記実装において 5. Test the model の項目を実行してください。また、自身で用意した画像に対しても文章を生成してください。これらに対してどういった文章が出力されたかを記録して提出してください。


データセットからの学習は行わず、学習済みの重みをダウンロードして利用します。


注意点として、デフォルトで設定されている重みのファイル名と、ダウンロードできる重みのファイル名は異なっています。ここは書き換える必要があります。



In [57]:
!python /Users/yuki.tatsuoka/diveintocode/week13~15課題/week14_課題/pytorch-tutorial/tutorials/03-advanced/image_captioning/sample.py  --image '/Users/yuki.tatsuoka/Desktop/IMG_4802.JPG'


<start> a group of stuffed animals in a room . <end>


In [75]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('/Users/yuki.tatsuoka/Desktop/IMG_4802.JPG')
cv2.imshow('dorakue',img)
cv2.waitKey(0)

107

# 【問題3】Kerasで動かしたい場合はどうするかを調査
PyTorchによる実装を動かしましたが、何らかの理由からKerasで動かしたい状況が考えられます。どういった手順を踏むことになるか調査し、できるだけ詳しく説明してください。


特に今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点については必ず触れてください。



In [ ]:
# MMdnnにて、pytorchからkerasに移行する事が可能。
# MMdnnはMicrosoft Researchにより開発が進められているオープンソースの深層学習モデルの変換と可視化を行うツール
# ちなみにkerasだけでなく、tensorflowにも使う事が可能
# 重みについては、VGG19, Resnet, MobileNetなどの事前学習済みの重みも使う事が可能。

# 具体的には
# mmdownload -f keras を実行すると、kerasで利用できる重みが出現する
# 例：Support frameworks: {'yolo2', 'inception_v3', 'xception', 'nasnet', 'densenet', 'vgg19', 'mobilenet', 'resnet50', 'vgg16', 'inception_resnet_v2'}

# また、重みを保存する場合
# mmdownload -f keras -n inception_v3 -o ./ にて保存する事が可能。
# 重みを他でも利用する事が可能なので、互換性を持たせて利用する